In [1]:
pip install pandas splink


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl
from splink.duckdb.blocking_rule_library import block_on
from splink.datasets import splink_datasets

In [3]:
df_left = pd.read_csv('/Users/rileyxiong/Downloads/group-project-spring-2024 (1)/Data/GroupProject/entity-resolution/left_dataset.csv')
df_right = pd.read_csv('/Users/rileyxiong/Downloads/group-project-spring-2024/Data/GroupProject/entity-resolution/right_dataset.csv')
print(df_left)
print(df_right)

       entity_id                              name  \
0              1                     The UPS Store   
1              2                St Honore Pastries   
2              3          Perkiomen Valley Brewery   
3              4                    Sonic Drive-In   
4              5                   Famous Footwear   
...          ...                               ...   
94580      94581     Adelita Taqueria & Restaurant   
94581      94582  INSPcenter/Thai Clinical Massage   
94582      94583              Wild Birds Unlimited   
94583      94584                 Claire's Boutique   
94584      94585                           Sic Ink   

                                          address          city state  \
0                 87 Grasso Plaza Shopping Center        Affton    MO   
1                                     935 Race St  Philadelphia    PA   
2                                   101 Walnut St    Green Lane    PA   
3                                   615 S Main St  Ashland 

In [4]:
# Rename 'zip_code' column in df_right to 'postal_code' for standardization

df_right.rename(columns={
    'zip_code': 'postal_code'  # Standardize more columns
}, inplace=True)


In [ ]:
# Display the first 5 rows of each dataset to verify changes

In [5]:
df_right.head(5)

,business_id,name,address,city,state,postal_code,size
0,1,SOURINI PAINTING INC.,12800 44th St N,Clearwater,FL,33762-4726,11.0
1,2,WOLFF DOLLA BILL LLC,1905 E 19th Ave,Tampa,FL,33605-2700,8.0
2,3,"COMPREHENSIVE SURGERY CENTER, LLC","1988 GULF TO BAY BLVD, Ste 1",CLEARWATER,FL,33765-3550,8.0
3,4,FRANK & ADAM APPAREL LLC,13640 Wright Cir,Tampa,FL,33626-3030,12.0
4,5,MORENO PLUS TRANSPORT INC,8608 Huron Court unite 58,Tampa,FL,33614,8.0


In [6]:
df_left.head(5)

,entity_id,name,address,city,state,postal_code
0,1,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123.0
1,2,St Honore Pastries,935 Race St,Philadelphia,PA,19107.0
2,3,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054.0
3,4,Sonic Drive-In,615 S Main St,Ashland City,TN,37015.0
4,5,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144.0


In [7]:
# Define a function to preprocess data for consistency and clean text fields

import re

def preprocess_data(df):
    # Convert postal codes and states to strings, ensuring no type discrepancies
    df['postal_code'] = df['postal_code'].apply(lambda x: str(int(float(x))) if ('.' in str(x)) else str(x))
    df['state'] = df['state'].astype(str).str.strip()
    df['name'] = df['name'].astype(str).str.lower().str.strip().apply(lambda x: re.sub(r'[^\w\s]', '', x))
    df['address'] = df['address'].astype(str).str.lower().str.strip().apply(lambda x: re.sub(r'[^\w\s]', '', x))
    df['city'] = df['city'].astype(str).str.lower().str.strip().apply(lambda x: re.sub(r'[^\w\s]', '', x))
    return df

# Apply the preprocessing function to both dataframes

df_left = preprocess_data(df_left)
df_right = preprocess_data(df_right)

In [8]:
# Update column names to unify id columns across datasets

df_left.rename(columns={'entity_id': 'unique_id'}, inplace=True)
df_right.rename(columns={'business_id': 'unique_id'}, inplace=True)

In [ ]:
# Standardize postal code format by removing any suffix after '-'

df_right['postal_code'] = df_right['postal_code'].str.split('-').str[0]
df_right = df_right.drop('size', axis=1)

In [9]:
df_right.head(5)

,unique_id,name,address,city,state,postal_code
0,1,sourini painting inc,12800 44th st n,clearwater,FL,33762
1,2,wolff dolla bill llc,1905 e 19th ave,tampa,FL,33605
2,3,comprehensive surgery center llc,1988 gulf to bay blvd ste 1,clearwater,FL,33765
3,4,frank adam apparel llc,13640 wright cir,tampa,FL,33626
4,5,moreno plus transport inc,8608 huron court unite 58,tampa,FL,33614


In [45]:
df_left.head(5)

,unique_id,name,address,city,state,postal_code
0,1,the ups store,87 grasso plaza shopping center,affton,MO,63123
1,2,st honore pastries,935 race st,philadelphia,PA,19107
2,3,perkiomen valley brewery,101 walnut st,green lane,PA,18054
3,4,sonic drivein,615 s main st,ashland city,TN,37015
4,5,famous footwear,8522 eager road dierbergs brentwood point,brentwood,MO,63144


In [10]:
print("Missing values in left_df:")
print(df_left.isnull().sum())

print("Missing values in right_df:")
print(df_right.isnull().sum())

Missing values in left_df:
unique_id      0
name           0
address        0
city           0
state          0
postal_code    0
dtype: int64
Missing values in right_df:
unique_id      0
name           0
address        0
city           0
state          0
postal_code    0
dtype: int64


In [11]:
from splink.duckdb.linker import DuckDBLinker
from splink.duckdb.blocking_rule_library import block_on
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl

# Define settings for data linking using Splink

settings = {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": [
        block_on("state"),
        block_on("postal_code"),
        block_on("city"),
        block_on(["substr(address, 1, 1)", "substr(name, 1, 1)", "substr(postal_code, 1, 3)"])
    ],
    "comparisons": [
        ctl.name_comparison("name"),
        ctl.name_comparison("address",
                            term_frequency_adjustments=True,
                            levenshtein_thresholds=[2],  
                            damerau_levenshtein_thresholds=[],
                            jaro_winkler_thresholds=[0.9, 0.8],  
                            jaccard_thresholds=[0.75]  
                            ),
    ],
}

# Initialize a DuckDBLinker object with defined settings

linker = DuckDBLinker([df_left, df_right], settings, input_table_aliases=["df_left", "df_right"])

In [12]:
# Display completeness chart for selected columns

linker.completeness_chart(cols=["name", "address", "postal_code"])

alt.LayerChart(...)

In [13]:
# Define deterministic rules for matching

deterministic_rules = [
    "l.name = r.name and levenshtein(r.postal_code, l.postal_code) <= 1",
    "l.address = r.address and levenshtein(r.postal_code, l.postal_code) <= 1",
    "l.name = r.name and levenshtein(r.address, l.address) <= 2"
]

# Re-estimate the probability of two random records matching
linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)


Probability two random records match is estimated to be  4.22e-06.
This means that amongst all possible pairwise record comparisons, one in 236,946.32 are expected to match.  With 8,682,051,735 total possible comparisons, we expect a total of around 36,641.43 matching pairs


In [14]:
linker.estimate_u_using_random_sampling(max_pairs=1e7, seed=1)

----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - name (no m values are trained).
    - address (no m values are trained).


In [15]:
session_name = linker.estimate_parameters_using_expectation_maximisation(block_on("name"))
session_address = linker.estimate_parameters_using_expectation_maximisation(block_on("address"))


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."name" = r."name"

Parameter estimates will be made for the following comparison(s):
    - address

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - name

Iteration 1: Largest change in params was -0.509 in the m_probability of address, level `Exact match address`
Iteration 2: Largest change in params was -0.000364 in the m_probability of address, level `Jaro_winkler_similarity >= 0.8`
Iteration 3: Largest change in params was -7.05e-06 in the m_probability of address, level `Jaccard >= 0.75`

EM converged after 3 iterations

Your model is not yet fully trained. Missing estimates for:
    - name (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."address" = r."address"

Parameter estimates will be made for the following comparison(s):
 

In [ ]:
results = linker.predict(threshold_match_probability=0.10)